# Pregunta 1, parte 1

## Importacion

In [1]:
import pandas as pd
from paths import *
from functools import reduce

In [2]:
# Documentación del módulo 'paths'.
# import paths
# help(paths)

## Carga de datos

### Creación de la tabla mezclada

In [3]:
# Cargar todos los csv en dos listas distinguidas por 
# la aparicion de 'furnished' en el nombre

# Lista de DataFrames (Sin 'furnished' en el nombre)
L_DF_a = [pd.read_csv(dict_csv_mc_a[wNN]) for wNN in L_WNN]
# Lista de DataFrames (Con 'furnished' en el nombre)
L_DF_f = [pd.read_csv(dict_csv_mc_f[wNN]) for wNN in L_WNN]

In [4]:
# se unen todos los data_frame en cada lista con el comando concat
DF_a = pd.concat(L_DF_a)
DF_f = pd.concat(L_DF_f)

# Se crea una columna con tantos ceros como la cantidad de filas de DF_a
# y con tantos unos como la cantidad de filas de DF_f 
furnished_col = pd.DataFrame(data = {'furnished':[0]*(DF_a.shape[0]) + [1]*(DF_f.shape[0])})

# Concateno los data frames DF_a y DF_f y le agrego al final la columna furnished_col
df = pd.concat([DF_a,DF_f], ignore_index=True) 
df = pd.concat([df,furnished_col], axis=1) 

In [5]:
# Quitamos las filas que sean iguales en todas las columnas con excepcion de
# la columna furnished y reindexamos
df.drop_duplicates(subset = df.columns[:-1], ignore_index=True, inplace=True)

# comentario: si hay filas duplicadas con distintos valores de furnished, se elimina la primera

In [6]:
# reportar si Reporte si existen observaciones de archivos con texto ’furnished’ que no est´en
# contenidos en archivos con texto ’all’

## Pregunta 1, parte 2

### Limpieza de datos

In [7]:
# Creamos valores a reemplazar
valores_nulos = {
    'property_type|rent_type|location':'Sin nombre',
    'price':'$0',
    'n_rooms':'NA',
    'n_bath':'NA',
    'surface':'0.0 m2',
    'details':'',
    'url':'',
    'metrocuadrado_index':0.0,
    'furnished':0
}

In [8]:
# Reemplazamos los valores
df.fillna(value=valores_nulos, inplace=True)
df.head()

,property_type|rent_type|location,price,n_rooms,n_bath,surface,details,url,metrocuadrado_index,furnished
0,"Casa en Arriendo, LA SOLEDAD NORTE Bogotá D.C..",$1.050.000,3,2,63.0 m2,Excelente casa cerca de la calle 80 y Avenida ...,https://www.metrocuadrado.com/inmueble/arriend...,25.6,0
1,"Casa en Arriendo, CIUDAD SALITRE SUR-ORIENTAL ...",$3.930.000,3,2,100.0 m2,Se arrienda casa bonita totalmente amoblada co...,https://www.metrocuadrado.com/inmueble/arriend...,57.3,0
2,"Casa en Arriendo, VILLA DE LOS ALPES II SECTOR...",$750.000,3,1,90.0 m2,CASA DE TRES NIVELES TRES ALCOBAS SALA COMEDOR...,https://www.metrocuadrado.com/inmueble/arriend...,12.2,0
3,"Casa en Arriendo, PARIS GAITAN Bogotá D.C..",$3.400.000,5,4,71.0 m2,casa ezquinera excelente ubicacion cinco alcob...,https://www.metrocuadrado.com/inmueble/arriend...,56.0,0
4,"Casa en Arriendo, BALCONES DE ORIENTE Bogotá D...",$1.560.000,4,3,96.0 m2,"Excelente casa de tres niveles, buena ubicació...",https://www.metrocuadrado.com/inmueble/arriend...,47.1,0


In [9]:
# prueba

#### Esquema para transformar precio a número

In [10]:
def precio_to_int(s:str)->int:
    """
    Toma un precio en forma de `str`, en el formato `'$XXX.XXX.XXX'` 
    con X algún número, y retorna el número entero de ese precio.
    """
    return int(s.replace('$', '').replace('.', ''))
    


#### Esquema para encontrar el número de garajes

In [11]:
# def n_garaje(s:str)->int: 
#     """
#     Función que toma una url en forma de `str` y retorna el número 
#     de garajes para la vivienda de esa url.
#     """
#     indice_garaje = s.find('-garajes')
    
#     if indice_garaje is -1: 
#         return 0
# #     print(s[indice_garaje - 1])
#     return int(s[indice_garaje - 1])
def n_garaje(s:str)->str: 
    """
    Función que toma una url en forma de `str` y retorna el número 
    de garajes para la vivienda de esa url.
    """
    i_garaje = s.find('-garajes')
    
    # No se encuentra el garaje
    if i_garaje is -1: 
        return '0'
    
    s_hasta_garaje = s[:i_garaje]
    i_garaje_ = s_hasta_garaje.rfind('-') + 1
    s_hasta_garaje[i_garaje_:]
    
    return s_hasta_garaje[i_garaje_:]

In [12]:
def n_surface(s:str)->float:
    """
    Toma un `str` en formato "XX.Xm2" y retorna el número real de los metros cuadrados
    """
    return float(s.replace('m2', ''))

n_surface( df['surface'][0])

63.0

In [21]:
def agregar_p_m2(x:float)->str:
    """
    Agrega "$/m2" al número que se le entregue
    """
    return "%.2f"%x + " $/m2"

In [23]:
col_precios = pd.Series(df['price']).map(precio_to_int)
col_superficie = pd.Series(df['surface']).map(n_surface)
col_p_entre_s = col_precios / col_superficie
col_p_entre_s_ = pd.Series(col_p_entre_s).map(agregar_p_m2)
col_garajes = pd.Series(df['url']).map(n_garaje)
df = pd.concat([df,col_p_entre_s_], axis=1) 
df = pd.concat([df,col_garajes], axis=1) 
# df
df

,property_type|rent_type|location,price,n_rooms,n_bath,surface,details,url,metrocuadrado_index,furnished,0,url
0,"Casa en Arriendo, LA SOLEDAD NORTE Bogotá D.C..",$1.050.000,3,2,63.0 m2,Excelente casa cerca de la calle 80 y Avenida ...,https://www.metrocuadrado.com/inmueble/arriend...,25.6,0,16666.67 $/m2,1
1,"Casa en Arriendo, CIUDAD SALITRE SUR-ORIENTAL ...",$3.930.000,3,2,100.0 m2,Se arrienda casa bonita totalmente amoblada co...,https://www.metrocuadrado.com/inmueble/arriend...,57.3,0,39300.00 $/m2,1
2,"Casa en Arriendo, VILLA DE LOS ALPES II SECTOR...",$750.000,3,1,90.0 m2,CASA DE TRES NIVELES TRES ALCOBAS SALA COMEDOR...,https://www.metrocuadrado.com/inmueble/arriend...,12.2,0,8333.33 $/m2,0
3,"Casa en Arriendo, PARIS GAITAN Bogotá D.C..",$3.400.000,5,4,71.0 m2,casa ezquinera excelente ubicacion cinco alcob...,https://www.metrocuadrado.com/inmueble/arriend...,56.0,0,47887.32 $/m2,0
4,"Casa en Arriendo, BALCONES DE ORIENTE Bogotá D...",$1.560.000,4,3,96.0 m2,"Excelente casa de tres niveles, buena ubicació...",https://www.metrocuadrado.com/inmueble/arriend...,47.1,0,16250.00 $/m2,1
...,...,...,...,...,...,...,...,...,...,...,...
18389,"Apartamento en Arriendo, CHICO NORTE Bogotá D.C..",$4.620.000,2,3,150.0 m2,SE ARRIENDA HERMOSO APARTAMENTO AMOBLADO DE 15...,https://www.metrocuadrado.com/inmueble/arriend...,55.6,1,30800.00 $/m2,2
18390,"Apartamento en Arriendo, CHICO NORTE II Bogotá...",$2.700.000,2,2,164.34 m2,Espectacular apartamento en exclusiva zona res...,https://www.metrocuadrado.com/inmueble/arriend...,26.1,1,16429.35 $/m2,1
18391,"Apartamento en Venta Y Arriendo, CHICO NORTE B...",$3.600.000,2,3,110.0 m2,Espectacular apartamento DUPLEX muy iluminado ...,https://www.metrocuadrado.com/inmueble/venta-a...,50.8,1,32727.27 $/m2,2
18392,"Apartamento en Arriendo, MOLINOS NORTE Bogotá ...",$2.525.000,2,3,106.0 m2,Hermoso apartamento de 106m2 2 añoS de const...,https://www.metrocuadrado.com/inmueble/arriend...,44.4,1,23820.75 $/m2,2
